In [7]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a") 

llm = HuggingFacePipeline.from_model_id(
    model_id = "gpt2",
    task = "text-generation",
    pipeline_kwargs = {"max_new_tokens": 50},
)

# llm.client.api_url = 'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1'

chain = prompt | llm

chain.invoke({
    "word" : "tomato"
})

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' plant grown for its high nutrient value, and it includes a variety of insects from many kinds of plants and plants with strong nutritional value including cucumbers, lettuce, cherry tomatoes, cabbage, carrots, corn and other plants not often found in Europe, tomatoes'

In [4]:
from langchain.llms import gpt4all
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a") 

llm = gpt4all(
    model="/Applications/gpt4all",
)

# llm.client.api_url = 'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1'

chain = prompt | llm

chain.invoke({
    "word" : "tomato"
})

TypeError: 'module' object is not callable

In [9]:
import json
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import streamlit as st
from langchain.retrievers import WikipediaRetriever
from langchain.schema import BaseOutputParser, output_parser

class JsonOutputParser(BaseOutputParser):
    def parse(self, text):
        text = text.replace("```", "").replace("json", "")
        return text



output_parser = JsonOutputParser()

function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo-1106",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
).bind(
    function_call={"name": "create_quiz"},
    functions=[function],
)

def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)

questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    You are a helpful assistant that is role playing as a teacher.
         
    Based ONLY on the following context make 10 (TEN) questions to test the user's knowledge about the text.
    
    Each question should have 4 answers, three of them must be incorrect and one should be correct.
         
    Use (o) to signal the correct answer.
         
    Context: {context}
""",
        )
    ]
)

questions_chain = {"context": format_docs} | questions_prompt | llm

In [10]:
retriever = WikipediaRetriever(top_k_results=5)
docs = retriever.get_relevant_documents("어린왕자")

In [7]:
docs

[Document(page_content='Seo Eun-kwang (Korean: 서은광; born November 22, 1990), also simply known as Eunkwang, is a South Korean singer, songwriter, entertainer and musical actor. He is the leader and main vocalist of the South Korean boy group BtoB.\n\n\n== Biography ==\nSeo was born on November 22, 1990, in Yongin, Gyeonggi in South Korea. He graduated from Dongshin University, majoring in practical music. He has one younger brother named Seo Eun-chong.\n\n\n=== Military service ===\nSeo reportedly began military duty by enlisting as an active duty soldier in the military band on August 21, 2018, at the ROK army\'s 27th division, nicknamed the \'We Will Win\' unit in Gangwon Province\'s Hwacheon County. In October 2018, he received first place out of the recruits in the overall results during basic training. On March 14, he was selected as ROK-SWC for receiving more than 90 points in sit-ups, push-ups, 3 km running, 10 km full combat gear marching and more.\nHe was discharged from the a

In [11]:
chain = questions_chain | output_parser
response = chain.invoke(docs)
response

''

In [ ]:
import json
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import streamlit as st
from langchain.retrievers import WikipediaRetriever
from langchain.schema import BaseOutputParser, output_parser

In [ ]:
function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo-1106",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

def wiki_search(term):
    retriever = WikipediaRetriever(top_k_results=5)
    docs = retriever.get_relevant_documents(term)
    return docs

questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    You are a helpful assistant that is role playing as a teacher.
         
    Based ONLY on the following context make 10 (TEN) questions to test the user's knowledge about the text.
    
    Each question should have 4 answers, three of them must be incorrect and one should be correct.
         
    Use (o) to signal the correct answer.
         
    Context: {context}
""",
        )
    ]
)

chain = {"context": wiki_search} | 

In [3]:
content='' additional_kwargs={'function_call': {'name': 'create_quiz', 'arguments': '{"questions":[{"question":"Where did the apple tree originate?","answers":[{"answer":"Central Asia","correct":true},{"answer":"North America","correct":false},{"answer":"Europe","correct":false},{"answer":"South America","correct":false}]},{"question":"How many cultivars of apples are there?","answers":[{"answer":"Over 5,000","correct":false},{"answer":"Over 10,000","correct":false},{"answer":"Over 7,500","correct":true},{"answer":"Over 3,000","correct":false}]},{"question":"What is the word \'apple\' descended from?","answers":[{"answer":"Proto-Indo-European *h₂ébōl","correct":true},{"answer":"Old English ancestor æppel","correct":false},{"answer":"Proto-Germanic noun *aplaz","correct":false},{"answer":"Middle English expression appel of paradis","correct":false}]},{"question":"What is the shape of apple tree leaves?","answers":[{"answer":"Simple ovals","correct":false},{"answer":"Egg shaped","correct":false},{"answer":"Glabrescent","correct":false},{"answer":"Convolute","correct":true}]},{"question":"What is the fruit of the apple tree called?","answers":[{"answer":"Pome","correct":true},{"answer":"Berry","correct":false},{"answer":"Drupes","correct":false},{"answer":"Aggregate fruit","correct":false}]},{"question":"Where is Apple Inc. headquartered?","answers":[{"answer":"New York City","correct":false},{"answer":"Los Angeles","correct":false},{"answer":"Cupertino, California","correct":true},{"answer":"Seattle","correct":false}]},{"question":"Who founded Apple Inc.?","answers":[{"answer":"Bill Gates","correct":false},{"answer":"Steve Jobs, Steve Wozniak, and Ronald Wayne","correct":true},{"answer":"Mark Zuckerberg","correct":false},{"answer":"Elon Musk","correct":false}]},{"question":"What is the name of the party game originally published by Out of the Box Publishing Inc.?","answers":[{"answer":"Cards Against Humanity","correct":false},{"answer":"Dixit","correct":false},{"answer":"Apples to Apples","correct":true},{"answer":"Monopoly","correct":false}]},{"question":"What is the basis of Mac, iPhone, iPad, and Apple Watch processors?","answers":[{"answer":"Intel processors","correct":false},{"answer":"ARM architecture","correct":true},{"answer":"AMD processors","correct":false},{"answer":"NVIDIA processors","correct":false}]},{"question":"What is the name of the family of SoCs used in the iPhone and certain iPad models?","answers":[{"answer":"B series","correct":false},{"answer":"C series","correct":false},{"answer":"A series","correct":true},{"answer":"D series","correct":false}]}]}'}}

SyntaxError: invalid syntax (960317023.py, line 1)